In [6]:
import pandas as pd
import geopandas as gpd

In [7]:
# dynamically get the world-country boundaries 
res = requests.get("https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json")
df = pd.DataFrame(json.loads(res.content.decode()))
df = df.assign(id=df["features"].apply(pd.Series)["id"],
         name=df["features"].apply(pd.Series)["properties"].apply(pd.Series)["name"])

In [8]:
# build a dataframe of country colours scraped from wikipedia
resp = requests.get("https://en.wikipedia.org/wiki/National_colours",)
soup = BeautifulSoup(resp.content.decode(), "html.parser")
colours = []
for t in soup.find_all("table", class_="wikitable"):
    cols = t.find_all("th")
    ok = (len(cols)>5 and cols[0].string.strip()=="Country" and cols[4].string.strip()=="Primary")
    if ok:
        for tr in t.find_all("tr"):
            td = tr.find_all("td")
            if len(td)>5:
                sp = td[4].find_all("span")
                c1 = re.sub("background-color:([\w,#,0-9]*).*", r"\1", sp[0]["style"])
                c2 = c1 if len(sp)==1 else re.sub("background-color:([\w,#,0-9]*).*", r"\1", sp[1]["style"])
                colours.append({"country":td[0].find("a").string, 
                                "colour1":c1,
                                "colour2":c2,

                               })
dfc = pd.DataFrame(colours).set_index("country")

In [23]:
countries_name = {
    'USA': "United States of America",
    'Tanzania': "United Republic of Tanzania",
    'Democratic Republic of Congo': "Democratic Republic of the Congo",
    'Congo': "Republic of the Congo",
    'Lao': "Laos",
    'Syrian Arab Republic': "Syria",
    'Serbia': "Republic of Serbia",
    'Czechia': "Czech Republic",
    'UAE': "United Arab Emirates",
}

for country_name in countries_name:
    df.replace(country_name, countries_name[country_name], inplace = True)

In [24]:
# a list of interesting countries - Singapore is missing!
countries = ["Singapore","Malaysia","Indonesia","Vietnam","Philippines"]
countries = list(df['name'])

In [14]:
# style the overlays with the countries own colors...
def style_fn(feature):
    cc = dfc.loc[feature["properties"]["name"]]
    ss= {'fillColor':f'{cc[0]}', 'color':f'{cc[1]}'}
    return ss

In [25]:
# create the base map
m = folium.Map(location = [1.34, 103.82],
               zoom_start=6,
               control_scale=True)

# overlay desired countries over folium map
for r in df.loc[df["name"].isin(countries)].to_dict(orient="records"):
    try:
        folium.GeoJson(
            r["features"], name=r["name"], tooltip=r["name"], style_function=style_fn
        ).add_to(m)
    except:
        print('error with: ', r['features']['properties']['name'])

m

error with:  Antarctica
error with:  French Southern and Antarctic Lands
error with:  The Bahamas
error with:  Democratic Republic of the Congo
error with:  Republic of the Congo
error with:  Falkland Islands
error with:  France
error with:  Guinea Bissau
error with:  Greenland
error with:  South Korea
error with:  Macedonia
error with:  New Caledonia
error with:  Papua New Guinea
error with:  Puerto Rico
error with:  North Korea
error with:  Western Sahara
error with:  Republic of Serbia
error with:  Swaziland
error with:  East Timor
error with:  United Republic of Tanzania
error with:  Vanuatu
error with:  West Bank
error with:  South Africa
